In [2]:
import pandas as pd
import numpy as np

# Capítulo 10 - Agregação de dados e operações em grupos

## 10.1 - Funcionamento do GroupBy

In [3]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [4]:
df

,key1,key2,data1,data2
0,a,one,-0.801432,0.167081
1,a,two,-0.112085,-0.494955
2,b,one,-1.019632,2.758031
3,b,two,-0.477242,3.317901
4,a,one,-1.678741,0.837174


In [5]:
grouped = df['data1'].groupby(df['key1'])

In [6]:
grouped

In [7]:
grouped.mean()

key1
a   -0.864086
b   -0.748437
Name: data1, dtype: float64

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [10]:
means

key1  key2
a     one    -1.240086
      two    -0.112085
b     one    -1.019632
      two    -0.477242
Name: data1, dtype: float64

In [11]:
means.unstack()

key2,one,two
key1,,
a,-1.240086,-0.112085
b,-1.019632,-0.477242


In [12]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [13]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [14]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.112085
            2006   -1.019632
Ohio        2005   -0.639337
            2006   -1.678741
Name: data1, dtype: float64

In [15]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.864086,0.169766
b,-0.748437,3.037966


In [16]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.240086  0.502127
     two  -0.112085 -0.494955
b    one  -1.019632  2.758031
     two  -0.477242  3.317901

In [17]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterando por grupos

In [18]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.801432  0.167081
1    a  two -0.112085 -0.494955
4    a  one -1.678741  0.837174
b
  key1 key2     data1     data2
2    b  one -1.019632  2.758031
3    b  two -0.477242  3.317901


In [19]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -0.801432  0.167081
4    a  one -1.678741  0.837174
a two
  key1 key2     data1     data2
1    a  two -0.112085 -0.494955
b one
  key1 key2     data1     data2
2    b  one -1.019632  2.758031
b two
  key1 key2     data1     data2
3    b  two -0.477242  3.317901


In [20]:
pieces = dict(list(df.groupby('key1')))

In [21]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.019632,2.758031
3,b,two,-0.477242,3.317901


In [22]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [23]:
grouped = df.groupby(df.dtypes, axis=1)

In [25]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.801432  0.167081
1 -0.112085 -0.494955
2 -1.019632  2.758031
3 -0.477242  3.317901
4 -1.678741  0.837174
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecionando uma coluna ou um subconjunto de colunas

In [26]:
df.groupby('key1')['data1']

In [27]:
df['data1'].groupby(df['key1'])

In [28]:
df.groupby('key1')[['data2']]

In [29]:
df[['data2']].groupby(df['key1'])

In [30]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.502127
     two  -0.494955
b    one   2.758031
     two   3.317901

In [31]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [32]:
s_grouped

In [33]:
s_grouped.mean()

key1  key2
a     one     0.502127
      two    -0.494955
b     one     2.758031
      two     3.317901
Name: data2, dtype: float64

### Agrupando com dicionários e Series

In [34]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [35]:
people.iloc[2:3, [1, 2]] = np.nan

In [36]:
people

,a,b,c,d,e
Joe,-0.539475,-0.590197,0.107399,0.115806,-1.438788
Steve,-0.456990,-0.239110,0.772404,1.593415,-0.775966
Wes,-1.998428,NaN,NaN,-1.901090,0.625290
Jim,-0.410705,0.845713,-0.098777,0.514459,-0.380805
Travis,-1.665969,-0.464890,0.611763,0.847555,-1.108363


In [37]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red'}

In [38]:
by_column = people.groupby(mapping, axis=1)

In [39]:
by_column.sum()

,blue,red
Joe,0.223205,-2.568460
Steve,2.365819,-1.472066
Wes,-1.901090,-1.373138
Jim,0.415681,0.054203
Travis,1.459318,-3.239223


In [40]:
map_series = pd.Series(mapping)

In [41]:
map_series

a     red
b     red
c    blue
d    blue
e     red
dtype: object

In [42]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Agrupando com funções

In [43]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.948608,0.255516,0.008621,-1.270825,-1.194302
5,-0.456990,-0.239110,0.772404,1.593415,-0.775966
6,-1.665969,-0.464890,0.611763,0.847555,-1.108363


In [44]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [45]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.998428 -0.590197  0.107399 -1.901090 -1.438788
  two -0.410705  0.845713 -0.098777  0.514459 -0.380805
5 one -0.456990 -0.239110  0.772404  1.593415 -0.775966
6 two -1.665969 -0.464890  0.611763  0.847555 -1.108363

### Agrupando por níveis de índice

In [46]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

In [47]:
hier_df = pd.DataFrame(np.random.randn(4, 5),
                       columns=columns)

In [48]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.232901 -0.851285 -0.459465 -1.070641 -0.558533
1     -1.591994 -0.742040 -1.024830 -0.510559  0.532111
2      0.598642 -0.352418  0.654875 -1.524786 -1.675389
3     -1.988613  1.371510  1.422356 -1.890387 -0.890770

In [49]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 - Agregação de dados